## Coroutines

    - pep-0342
    - generators are data producers
    - coroutines are data consumers

    - Coroutines consume values using a (yield)

In [1]:
def hello():  # -- generator
    # print('Hello world')
    # return 'Hello world'
    yield "Hello world"


hello()

<generator object hello at 0x7f61d0801000>

In [2]:
result = hello()
print(f"{type(result)} {result}")  # generator
print(f"{next(result) = }")  # 'Hello world'

<class 'generator'> <generator object hello at 0x7f61d08010e0>
next(result) = 'Hello world'


In [3]:
def hello():  # generator
    value = "Hello world"
    yield value


hello()

<generator object hello at 0x7f61d0800a50>

In [4]:
result = hello()
print(f"{type(result)} {result}")  # generator
print(f"{next(result) = }")  # 'Hello world'

<class 'generator'> <generator object hello at 0x7f61d0801070>
next(result) = 'Hello world'


In [5]:
def hello():  # coroutine
    value = yield "Hello world"
    yield value


result = hello()
print(f"{type(result)} {result}")  # generator
print(f"{next(result) = }")  # 'Hello world'

<class 'generator'> <generator object hello at 0x7f61d0800d60>
next(result) = 'Hello world'


In [6]:
def hello():  # coroutine
    value = yield
    yield value


# NOTE: Default initial yield is None

result = hello()
print(f"{type(result)} {result}")
print(f"{next(result) = }")  # None

<class 'generator'> <generator object hello at 0x7f61d0801700>
next(result) = None


# # Anothr Examples

In [7]:
def my_coroutine(num):
    value = yield "default string"
    yield value
    yield value
    value = yield "new string"
    yield value

In [8]:
# Step 1: To call
c = my_coroutine(4)
print(f"{type(c)    = }")  # <class 'generator'>
print(f"{c          = }")  # <generator object my_coroutine at 0x0000022D8B349A10>

type(c)    = <class 'generator'>
c          = <generator object my_coroutine at 0x7f61d0801850>


In [9]:
# Step 2: to prime the generator to become coroutine
print(f"{next(c) =}")  # 'default string'

next(c) ='default string'


In [10]:
# Step 3: Sending values to coroutine
print(f"{c.send('first')  =}")  # 'first'
print(f"{c.send('second') =}")  # 'first'
print(f"{c.send('third')  =}")  # 'new string'

c.send('first')  ='first'
c.send('second') ='first'
c.send('third')  ='new string'


In [11]:
c.send("fourth")

'fourth'

In [13]:
print(f"{c.send('first')  =}")  

StopIteration: 

In [14]:
# To close the coroutine
c.close()

## Another Example with Infinite receive

In [15]:
def my_coroutine():
    while True:
        received = yield 1234  # values will be received here
        print(f"Received :{received}")


# Step 1: creating the generator
it = my_coroutine()

# Step 2: Prime the coroutine
print(f"{next(it) =}")  # 1234


next(it) =1234


In [16]:
# Step 3: sending values to coroutine
it.send("First")

Received :First


1234

In [17]:
it.send("Second")
it.send("third")

Received :Second
Received :third


1234

In [18]:
for i in range(9):
    it.send(i)

Received :0
Received :1
Received :2
Received :3
Received :4
Received :5
Received :6
Received :7
Received :8


In [19]:
# Step 4: close the coroutine
it.close()


In [20]:
try:
    it.send("fourth")
except StopIteration:
    print("coroutine is closed. Cant send any value")

coroutine is closed. Cant send any value


### Finite Reception

In [21]:
def my_coroutine(count=5):
    for _ in range(count):
        received = yield 1234  # values will be received here
        print(f"Received :{received}")

# Step 1: creating the generator
it = my_coroutine(5)

# Step 2: Prime the coroutine
print(f"{next(it) =}")


# Step 3: sending values to coroutine
it.send("First")
it.send("Second")
it.send("third")


next(it) =1234
Received :First
Received :Second
Received :third


1234

In [22]:
for i in range(9):
    it.send(i)


Received :0
Received :1


StopIteration: 

In [23]:
# Step 4: close the coroutine
it.close()

try:
    it.send("fourth")
except StopIteration:
    print("coroutine is closed. Cant send any value")


coroutine is closed. Cant send any value


##  chaining the coroutines

In [1]:
def producer(sentence, next_coroutine):
    """
    Producer which just split strings and
    feed it to pattern_filter coroutine
    """
    tokens = sentence.split(" ")
    for token in tokens:
        next_coroutine.send(token)
    next_coroutine.close()


def pattern_filter(pattern="ing", next_coroutine=None):
    """
    Search for pattern in received token
    and if pattern got matched, send it to
    print_token() coroutine for printing
    """
    print("Searching for {}".format(pattern))
    try:
        while True:
            token = yield
            if pattern in token:
                next_coroutine.send(token)
    except GeneratorExit:
        print("Done with filtering!!!")


def print_token():
    """
    Act as a sink, simply print the
    received tokens
    """
    print("I'm sink, i'll print tokens")
    try:
        while True:
            token = yield
            print(token)
    except GeneratorExit:
        print("Done with printing!")


In [4]:
# Pipeline

pt = print_token()
next(pt)

I'm sink, i'll print tokens


In [5]:
pf = pattern_filter(next_coroutine=pt)
next(pf)

Searching for ing


In [6]:
producer(
    sentence = "Bob is running behind a fast moving car", 
    next_coroutine=pf
)

running
moving
Done with filtering!!!


Bad pipe message: %s [b'Z\x0b\\\x84.6\xd7|(\xcc\x81\xadp4o=\xe5\xc0 \x1a)\\\xeb\xcc\x014\x15\x02\x14\x00p\x83h\x03\xabM[~m\xb2e\xba\xe1\xf7[?\xbd\x88\x03\xd4\xff\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 n\xdc\xbf[\xd6r\x8fB\xac\xb3\xf7D\x9c\xf3^\xe8 b\x9fd\xb4\xafx\xfc\x97\xdf\xd6\xbe\tk*']
Bad pipe message: %s [b"\x14\xed\xf9\xd5\x9c\xfd\xb1_e\xa5\xf1\xb6\xbe,\x9dL\xd3=\x00\x00\x86\xc00\xc0,\xc0(\xc0$\xc0\x14\xc0\n\x00\xa5\x00\xa3\x00\xa1\x00\x9f\x00k\x00j\x00i\x00h\x009\x008\x007\x006\xc02\xc0.\xc0*\xc0&\xc0\x0f\xc0\x05\x00\x9d\x00=\x005\xc0/\xc0+\xc0'\xc0#\xc0\x13\xc0\t\x00\xa4\x00\xa2\x00\xa